# <u>Bicycle Rental Management System Code Review and Enhancements</u>
## Objective
- This session of this project focuses  on developing and refining components of the Bicycle Rental Management System to enable efficient database management, bicycle search functionality, and purchase recommendations. This enables a  user-friendly search,select and rental interface with advanced filtering and suggestion features.

## SEARCHING FOR BIKE BUTTON

In [1]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
from database import DatabaseManager  # Ensure you have your database manager set up

# Create an instance of DatabaseManager
db_manager = DatabaseManager()

# Create the main "Search" button
search_button = widgets.Button(
    description="Search for a Bicycle",
    button_style='info',  # Optional styling
    tooltip="Click to search for bicycles",
    icon="search"  # Optional icon
)

# Define the function that runs when search_button is clicked
def on_search_click(b):
    clear_output()  # Clear previous output
    display(widgets.HTML("<h3>Search Bicycles</h3>"))  # Display the title

    # Create input fields for search criteria
    brand_input = widgets.Text(description="Brand")
    type_input = widgets.Text(description="Type")
    status_input = widgets.Dropdown(
        options=["Available", "Rented", "Under Maintenance"],
        description="Status"
    )
    condition_input = widgets.Dropdown(
        options=["New", "Good", "Fair", "Damaged"],
        description="Condition"
    )
    min_rate_input = widgets.IntText(description="Min Rate", value=0)
    max_rate_input = widgets.IntText(description="Max Rate", value=0)
    sort_by_input = widgets.Dropdown(
        options=["BRAND", "TYPE", "RENTAL_RATE", "CONDITION"],
        description="Sort By"
    )
    
    # Create a button to execute the search
    search_execute_button = widgets.Button(description="Execute Search")
    error_output = widgets.Output()  # Output for error messages
    display(error_output)

    # Define the function to perform the search
    def perform_search(e):
        brand = brand_input.value.strip()
        bike_type = type_input.value.strip()
        status = status_input.value.strip()
        condition = condition_input.value.strip()
        
        # Validate rate inputs
        min_rate = min_rate_input.value
        max_rate = max_rate_input.value
        
        with error_output:
            clear_output()  # Clear previous error messages
            if min_rate < 0:
                print("Error: Minimum rate cannot be negative.")
                return
            if max_rate < 0:
                print("Error: Maximum rate cannot be negative.")
                return
            if min_rate > max_rate and max_rate != 0:
                print("Error: Maximum rate must be greater than or equal to minimum rate.")
                return
            
            sort_by = sort_by_input.value
            
            # Execute the search query against the database
            results = db_manager.search_bicycles(
                brand=brand, 
                bike_type=bike_type, 
                status=status, 
                condition=condition, 
                min_rate=min_rate, 
                max_rate=max_rate, 
                sort_by=sort_by
            )

            # Display the search results
            clear_output()  # Clear output for a fresh display
            print("Searching with criteria:")
            print(f"Brand: {brand}, Type: {bike_type}, Status: {status}, Condition: {condition}")
            print(f"Min Rate: {min_rate}, Max Rate: {max_rate}, Sort By: {sort_by}")

            if not results.empty:
                print("\nAvailable Bicycles:")
                display(results)
            else:
                print("No exact matches found.")
                print("Would you like to see similar bicycles?")
                similar_bikes = db_manager.suggest_similar_bicycles(brand, bike_type, status, condition)
                if not similar_bikes.empty:
                    print("Similar bicycles found:")
                    display(similar_bikes)

    # Display all search fields and execution button
    display(brand_input, type_input, status_input, condition_input, 
            min_rate_input, max_rate_input, sort_by_input, search_execute_button)
    
    # Link the search execution button to the perform_search function
    search_execute_button.on_click(perform_search)

# Link the main search button to the on_search_click function
search_button.on_click(on_search_click)

# Display the main search button
display(search_button)



HTML(value='<h3>Search Bicycles</h3>')

Output()

Text(value='', description='Brand')

Text(value='', description='Type')

Dropdown(description='Status', options=('Available', 'Rented', 'Under Maintenance'), value='Available')

Dropdown(description='Condition', options=('New', 'Good', 'Fair', 'Damaged'), value='New')

IntText(value=0, description='Min Rate')

IntText(value=0, description='Max Rate')

Dropdown(description='Sort By', options=('BRAND', 'TYPE', 'RENTAL_RATE', 'CONDITION'), value='BRAND')

Button(description='Execute Search', style=ButtonStyle())

## RENTING BIKE BUTTON

In [ ]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import date, timedelta

# Create the main "Rent a Bicycle" button
rent_button = widgets.Button(
    description="Rent a Bicycle",
    button_style='success',
    tooltip="Click to rent a bicycle",
    icon="bicycle"
)

# Define the function that runs when rent_button is clicked
def on_rent_click(b):
    clear_output()  # Clear previous outputs
    display(widgets.HTML("<h3>Rent a Bicycle</h3>"))  # Display the section header

    # Input fields for renting a bicycle
    bicycle_id_input = widgets.IntText(description="Bicycle ID", placeholder="Enter Bicycle ID")
    member_id_input = widgets.IntText(description="Member ID", placeholder="Enter Member ID")
    rental_date_input = widgets.DatePicker(description="Rental Date", value=date.today())
    rental_duration_input = widgets.IntText(description="Rental Duration (Days)", value=1, min=1)  # Duration input
    rent_execute_button = widgets.Button(description="Rent")

    # Function to handle the rental logic
    def perform_rent(e):
        bicycle_id = bicycle_id_input.value
        member_id = member_id_input.value
        rental_date = rental_date_input.value
        rental_duration = rental_duration_input.value

        # Input validation
        if bicycle_id <= 0 or member_id <= 0:
            print("Error: Bicycle ID and Member ID must be positive integers.")
            return
        if rental_date is None:
            print("Error: Please select a valid rental date.")
            return

        # Check bicycle availability (you would normally check this against a database)
        if not is_bicycle_available(bicycle_id):
            print(f"Error: Bicycle ID {bicycle_id} is not available for rental.")
            return

        # Assuming a fixed rental rate for demonstration
        rental_rate = 25  # Example fixed rate per day
        total_cost = rental_rate * rental_duration

        # Here you would typically call a method from your BikeRent class
        # bike_rent_manager.rent_bike(bicycle_id, member_id, rental_date, rental_duration)
        
        # Provide feedback to the user
        print(f"Successfully rented Bicycle ID: {bicycle_id}")
        print(f"Member ID: {member_id}")
        print(f"Rental Date: {rental_date}")
        print(f"Rental Duration: {rental_duration} days")
        print(f"Total Cost: ${total_cost:.2f}")

        # Clear input fields after submission
        bicycle_id_input.value = 0
        member_id_input.value = 0
        rental_date_input.value = date.today()
        rental_duration_input.value = 1

    # Display input fields and button
    display(bicycle_id_input, member_id_input, rental_date_input, rental_duration_input, rent_execute_button)

    # Link the button click to the perform_rent function
    rent_execute_button.on_click(perform_rent)

# Dummy function to check bicycle availability
def is_bicycle_available(bicycle_id):
    # This function should query the database to check if the bicycle is available
    # For now, we'll simulate that all bicycles are available
    return True

# Link button click to the on_rent_click function
rent_button.on_click(on_rent_click)

# Display the rent button
display(rent_button)




HTML(value='<h3>Rent a Bicycle</h3>')

IntText(value=0, description='Bicycle ID')

IntText(value=0, description='Member ID')

DatePicker(value=datetime.date(2024, 11, 2), description='Rental Date', step=1)

IntText(value=1, description='Rental Duration (Days)')

Button(description='Rent', style=ButtonStyle())

## RETURNING BIKE BUTTON

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# adding CSS for styling
def apply_custom_css():
    style = """
    <style>
        .return-button {
            background-color: #ffcc00;  /* Bright yellow */
            color: black;                 /* Black text */
            font-weight: bold;
            border-radius: 5px;
            padding: 10px 15px;
            font-size: 16px;
        }
        .return-button:hover {
            background-color: #e6b800;   /* Darker yellow on hover */
        }
        .input-field {
            margin: 5px 0;               /* Space between input fields */
            width: 300px;                 /* Fixed width for input fields */
        }
        .confirmation-box {
            background-color: #f2f2f2;   /* Light gray background */
            border: 1px solid #ccc;      /* Border around the confirmation box */
            border-radius: 5px;
            padding: 15px;
            margin-top: 10px;
        }
        .confirmation-button {
            background-color: #4CAF50;   /* Green background */
            color: white;                 /* White text */
            padding: 10px 15px;
            border: none;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
        }
        .confirmation-button:hover {
            background-color: #45a049;    /* Darker green on hover */
        }
    </style>
    """
    display(HTML(style))

# Creating the main "Return Bicycle" button
return_button = widgets.Button(
    description="Return a Bicycle",
    button_style='',  # Use empty string to apply custom styling
    tooltip="Click to return a bicycle",
    icon="undo",
    layout=widgets.Layout(width='auto', height='40px'),
    style={'button_color': '#ffcc00'}  # Default button color
)

# Defining the function that runs when return_button is clicked
def on_return_click(b):
    clear_output()  # Clear previous outputs
    display(HTML("<h3>Return a Bicycle</h3>"))  # Display the section header

    # Inputing field for Bicycle ID
    bicycle_id_input = widgets.IntText(description="Bicycle ID", placeholder="Enter Bicycle ID", layout=widgets.Layout(width='300px'), style={'description_width': 'initial'})
    return_date_input = widgets.DatePicker(description="Return Date", layout=widgets.Layout(width='300px'), style={'description_width': 'initial'})  # Optional return date input
    condition_input = widgets.Dropdown(
        options=["Good", "Fair", "Damaged"],
        description="Condition",
        layout=widgets.Layout(width='300px'),
        style={'description_width': 'initial'}
    )
    damage_fee_input = widgets.FloatText(description="Damage Fee", value=0.0, layout=widgets.Layout(width='300px'), style={'description_width': 'initial'})  # Input for damage fee
    comments_input = widgets.Textarea(
        description="Comments",
        placeholder="Additional comments or notes about the return",
        layout=widgets.Layout(width='300px', height='100px'),
        style={'description_width': 'initial'}
    )
    return_execute_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(width='auto', height='40px'))  # Button to execute return action

    # Function to handle the confirmation of the return
    def confirm_return(e):
        confirmation_box = widgets.VBox([
            widgets.HTML("<b>Are you sure you want to return this bicycle?</b>"),
            widgets.Button(description="Confirm", button_style='success', layout=widgets.Layout(width='auto', height='40px')),
            widgets.Button(description="Cancel", button_style='danger', layout=widgets.Layout(width='auto', height='40px'))
        ], layout=widgets.Layout(border='solid 1px #ccc', padding='10px', width='320px', margin='10px 0'))

        display(confirmation_box)

        # Function to perform the return logic
        def perform_return_confirm(e):
            bicycle_id = bicycle_id_input.value
            return_date = return_date_input.value
            condition = condition_input.value
            damage_fee = damage_fee_input.value
            comments = comments_input.value
            
            # Here you would call the return method on the return_manager
            # For now, we are just printing the values
            print(f"Returning Bicycle ID: {bicycle_id}")
            print(f"Return Date: {return_date}")
            print(f"Condition: {condition}")
            print(f"Damage Fee: ${damage_fee:.2f}")
            print(f"Comments: {comments}")
            clear_output()  # Clear the input fields and confirmation box
            display(HTML("<h4>Bicycle returned successfully!</h4>"))  # Success message
        
        # Linking the  confirm button 
        confirmation_box.children[1].on_click(perform_return_confirm)  # Confirm button
        confirmation_box.children[2].on_click(lambda e: clear_output())  # Cancel button

    # Displaying input fields and button
    display(bicycle_id_input, return_date_input, condition_input, damage_fee_input, comments_input, return_execute_button)

    # Linking the button click to the confirm_return function
    return_execute_button.on_click(confirm_return)

# Linking  button click to the on_return_click function
return_button.on_click(on_return_click)

# Applying  CSS styles
apply_custom_css()

# Displaying the return button
display(return_button)


Button(description='Return a Bicycle', icon='undo', layout=Layout(height='40px', width='auto'), style=ButtonSt…

## SELECT BIKE FOR PURCHASE ORDER

In [8]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Define the BikeSelector class
class BikeSelector:
    def __init__(self):
        # Initialize any necessary attributes
        print("BikeSelector initialized.")
        
    def recommend_purchases(self, budget):
        # Placeholder for recommendation logic
        # In a real scenario, replace this with actual recommendation logic based on bike inventory and rental data
        print(f"Generating recommendations for budget: {budget}")
        if budget <= 100:
            return pd.DataFrame(columns=["Type", "Units", "Estimated Cost"])  # No recommendations if budget is too low
        else:
            return pd.DataFrame({"Type": ["Mountain Bike", "Road Bike"], "Units": [2, 3], "Estimated Cost": [200, 300]})

# Create an instance of BikeSelector
selector_manager = BikeSelector()

# Create the main "Select Bicycles for Purchase Order" button
purchase_button = widgets.Button(
    description="Select Bicycles for Purchase Order",
    button_style='info',  # Button style can be 'info', 'success', 'warning', etc.
    tooltip="Click to select bicycles for purchase order",
    icon="shopping-cart"
)

# Function to handle button clicks
def on_purchase_click(b):
    clear_output()
    display(HTML("<h3>Select Bicycles for Purchase Order</h3>"))

    # Input field for budget
    budget_input = widgets.IntText(description="Budget", placeholder="Enter your budget")
    purchase_execute_button = widgets.Button(description="Recommend Purchases")

    # Function to get purchase recommendations
    def perform_purchase(e):
        budget = budget_input.value
        if budget <= 0:
            print("Please enter a valid budget greater than zero.")
            return
        
        # Call the recommend_purchases method on the selector_manager
        recommendations = selector_manager.recommend_purchases(budget)
        
        if recommendations.empty:
            print("No recommendations available within your budget.")
        else:
            print("Recommendations for your budget:")
            display(recommendations)

    # Display input fields and button
    display(budget_input, purchase_execute_button)
    purchase_execute_button.on_click(perform_purchase)

# Link button click to the function
purchase_button.on_click(on_purchase_click)

# Display the purchase button
display(purchase_button)


BikeSelector initialized.


Button(button_style='info', description='Select Bicycles for Purchase Order', icon='shopping-cart', style=Butt…

# EXIT BIKE APPLICATION

In [9]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Define the BikeSelector class
class BikeSelector:
    def __init__(self):
        # Initialize any necessary attributes
        print("BikeSelector initialized.")
        
    def recommend_purchases(self, budget):
        # Placeholder for recommendation logic
        print(f"Generating recommendations for budget: {budget}")
        if budget <= 100:
            return pd.DataFrame(columns=["Type", "Units", "Estimated Cost"])  # No recommendations if budget is too low
        else:
            return pd.DataFrame({"Type": ["Mountain Bike", "Road Bike"], "Units": [2, 3], "Estimated Cost": [200, 300]})

# Create an instance of BikeSelector
selector_manager = BikeSelector()

# Create the main "Select Bicycles for Purchase Order" button
purchase_button = widgets.Button(
    description="Select Bicycles for Purchase Order",
    button_style='info',  # Button style can be 'info', 'success', 'warning', etc.
    tooltip="Click to select bicycles for purchase order",
    icon="shopping-cart"
)

# Function to handle button clicks for purchase
def on_purchase_click(b):
    clear_output()
    display(HTML("<h3>Select Bicycles for Purchase Order</h3>"))

    # Input field for budget
    budget_input = widgets.IntText(description="Budget", placeholder="Enter your budget")
    purchase_execute_button = widgets.Button(description="Recommend Purchases")

    # Function to get purchase recommendations
    def perform_purchase(e):
        budget = budget_input.value
        if budget <= 0:
            print("Please enter a valid budget greater than zero.")
            return
        
        # Call the recommend_purchases method on the selector_manager
        recommendations = selector_manager.recommend_purchases(budget)
        
        if recommendations.empty:
            print("No recommendations available within your budget.")
        else:
            print("Recommendations for your budget:")
            display(recommendations)

    # Display input fields and button
    display(budget_input, purchase_execute_button)
    purchase_execute_button.on_click(perform_purchase)

# Link purchase button click to the function
purchase_button.on_click(on_purchase_click)

# Create the main "Exit" button
exit_button = widgets.Button(
    description="Exit",
    button_style='danger',  # Button style for emphasis
    tooltip="Click to exit the application",
    icon="exit"
)

# Function to handle exit button click
def on_exit_click(b):
    clear_output()
    print("Thank you for using the Bicycle Rental Management System!")

# Link exit button click to the function
exit_button.on_click(on_exit_click)

# Display the purchase button and exit button
display(purchase_button, exit_button)


BikeSelector initialized.


Button(button_style='info', description='Select Bicycles for Purchase Order', icon='shopping-cart', style=Butt…

Button(button_style='danger', description='Exit', icon='exit', style=ButtonStyle(), tooltip='Click to exit the…